## Import Songs

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
## Data gathered from https://components.one/datasets/billboard-200/
songs = pd.read_csv('./data/acoustic_features.csv')
albums = pd.read_csv('./data/albums.csv')
top100 = pd.read_csv('./data/top100_by_year.csv').iloc[:, 1:]

In [3]:
songs.head()

,id,song,album,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,date
0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.0555,0.754,142301.0,0.663,0.000000,6.0,0.101,-6.311,0.0,0.427,90.195,4.0,0.207,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.2920,0.860,152829.0,0.418,0.000000,7.0,0.106,-9.061,0.0,0.158,126.023,4.0,0.374,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.1530,0.718,215305.0,0.454,0.000046,8.0,0.116,-9.012,1.0,0.127,89.483,4.0,0.196,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
3,1wJRveJZLSb1rjhnUHQiv6,Swervin (feat. 6ix9ine),Hoodie SZN,A Boogie Wit da Hoodie,0.0153,0.581,189487.0,0.662,0.000000,9.0,0.111,-5.239,1.0,0.303,93.023,4.0,0.434,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
4,0jAfdqv18goRTUxm3ilRjb,Startender (feat. Offset and Tyga),Hoodie SZN,A Boogie Wit da Hoodie,0.0235,0.736,192779.0,0.622,0.000000,6.0,0.151,-4.653,0.0,0.133,191.971,4.0,0.506,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21


In [4]:
#340,000 songs/observations
songs.shape

(339855, 19)

In [5]:
##Convert date to year
songs['date'] = pd.to_datetime(songs['date'])
songs['year'] = songs['date'].dt.year
songs.drop(columns=['album', 'album_id', 'date'], inplace=True)

In [6]:
##Drop songs that are newer than 2015 (the top 100 dataset only goes up to 2015)
new_songs = songs[songs['year']>2015]
songs.drop(new_songs.index, inplace=True)

In [7]:
## Top 100 by Year looks good, but we need to give each observation a song ID
## So that we can compare with the larger song table
top100[top100['Year']==1965].head()

,Song Title,Artist,Year,Position
1080,Wooly Bully,Sam The Sham and The Pharaohs,1965,1
1081,I Can't Help Myself (Sugar Pie Honey Bunch),Four Tops,1965,2
1082,(I Can't Get No) Satisfaction,Rolling Stones,1965,3
1083,You Were On My Mind,We Five,1965,4
1084,You've Lost That Lovin' Feelin',Righteous Brothers,1965,5


## Spotify

In [8]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

market = [ "AD", "AR", "AT", "AU", "BE", "BG", "BO", "BR", "CA", "CH", "CL", "CO", "CR", "CY", 
      "CZ", "DE", "DK", "DO", "EC", "EE", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HU", 
      "ID", "IE", "IS", "IT", "JP", "LI", "LT", "LU", "LV", "MC", "MT", "MX", "MY", "NI", "NL", 
      "NO", "NZ", "PA", "PE", "PH", "PL", "PT", "PY", "SE", "SG", "SK", "SV", "TH", "TR", "TW", 
      "US", "UY", "VN" ]

CLIENT_ID = "07663e5a63794b58b412cdbe45565d1a"
CLIENT_SECRET = "991bcea116134ab29fe3793adb25b648"

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)

In [9]:
## Going to use NLTK to clean up each song's title and artist to make it easier to search with the Spotify API
# Load library
from nltk.corpus import stopwords

# You will have to download the set of stop words the first time
import nltk
nltk.download('stopwords')

# Load stop words
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jcacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
## This function will clean the song titles and artist names, 
## removing non-alphanumeric characters and things like featured artists
def clean_text(text):
    #Make lower-case
    text = text.lower()
    #Keep only the first name if there are two names separated by forward-slash
    if text.find('/') > 0:
        delimiter = text.find('/')
        text = text[:delimiter-1]
    if text.find('feat')>0:
        delimiter = text.find('feat')
        text = text[:delimiter-1]
    #Remove non-alphanumeric characters
    text = re.sub(r'([^\s\w]|_)+', '', text)
    #Remove stop-words
    tokenized_name = text.split(" ")
    filtered_tokens = [word for word in tokenized_name if word not in stop_words and len(word)>1]
    text = " ".join(filtered_tokens)
    return text

In [11]:
## Testing the function
clean_text('Post To Be | Omarion feat. Chris Brown and Jhe')

'post omarion'

In [12]:
## This query will use different variations of the song name and artist name to query the Spotify API and
## increase the likelihood of obtaining the correct song
def get_song_details(query):
    song_name = query.split('|')[0]
    song_name = clean_text(song_name)
    try:
        artist = query.split('|')[1]
        artist = clean_text(artist)
    except:
        artist = ""

    try:
        results = sp.search(q=query, type='track')
        artist = results['tracks']['items'][0]['artists'][0]['name']
        song_name = results['tracks']['items'][0]['name']
        song_id = results['tracks']['items'][0]['id']
        return artist, song_name, song_id
    except:
        try:
            results = sp.search(q=clean_name+" "+artist, type='track')
            artist = results['tracks']['items'][0]['artists'][0]['name']
            song_name = results['tracks']['items'][0]['name']
            song_id = results['tracks']['items'][0]['id']
            return artist, song_name, song_id
        except:
            try:
                results = sp.search(q=song_name, type='track')
                artist = results['tracks']['items'][0]['artists'][0]['name']
                song_name = results['tracks']['items'][0]['name']
                song_id = results['tracks']['items'][0]['id']
                return artist, song_name, song_id
            except:
                try:
                    results = sp.search(q=short_name, type='track')
                    artist = results['tracks']['items'][0]['artists'][0]['name']
                    song_name = results['tracks']['items'][0]['name']
                    song_id = results['tracks']['items'][0]['id']
                    return artist, song_name, song_id
                except:
                    return np.nan

In [13]:
## Testing the function
get_song_details('post omarion')

('Omarion',
 'Post to Be (feat. Chris Brown & Jhene Aiko)',
 '0fgZUSa7D7aVvv3GfO0A1n')

In [14]:
## Testing the function
get_song_details("sugar shack jimmy")

('Jimmy Gilmer & The Fireballs', 'Sugar Shack', '2nLLenueHlqs60IcDn9lan')

### Now lets add Song ID to our Top100 dataframe

In [15]:
## Removing songs from prior to 1963 because our larger song dataset starts at 1963
top100 = top100[top100['Year'] > 1962]
top100.shape

(5300, 4)

In [16]:
## Reset index after removals
top100.reset_index(drop=True, inplace=True)

In [17]:
top100.head()

,Song Title,Artist,Year,Position
0,Sugar Shack,Jimmy Gilmer and The Fireballs,1963,1
1,Surfin' U.S.A.,Beach Boys,1963,2
2,The End Of The World,Skeeter Davis,1963,3
3,Rhythm Of The Rain,Cascades,1963,4
4,He's So Fine,Chiffons,1963,5


In [18]:
## Testing get_song_details
get_song_details(top100['Song Title'][5223] + " | " + top100['Artist'][5223])

('Omarion',
 'Post to Be (feat. Chris Brown & Jhene Aiko)',
 '0fgZUSa7D7aVvv3GfO0A1n')

In [19]:
## Grabbing the acoustic features of that particular song (test)
pd.DataFrame(sp.audio_features('0fgZUSa7D7aVvv3GfO0A1n'))

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0697,https://api.spotify.com/v1/audio-analysis/0fgZ...,0.733,226581,0.676,0fgZUSa7D7aVvv3GfO0A1n,0,10,0.208,-5.655,0,0.0432,97.448,4,https://api.spotify.com/v1/tracks/0fgZUSa7D7aV...,audio_features,spotify:track:0fgZUSa7D7aVvv3GfO0A1n,0.701


In [21]:
top100['Song Title'][5223] + " " + top100['Artist'][5223]

'Post To Be Omarion feat. Chris Brown and Jhene Aiko'

In [22]:
top100['Query'] = top100['Song Title'] + " | " + top100['Artist']

In [23]:
top100[top100['Artist']=='Omarion feat. Chris Brown and Jhene Aiko']['Query']

5223    Post To Be | Omarion feat. Chris Brown and Jhe...
Name: Query, dtype: object

In [24]:
test_df = top100[5247:5296]

In [25]:
test_df.head()

,Song Title,Artist,Year,Position,Query
5247,B**** Better Have My Money,Rihanna,2015,48,B**** Better Have My Money | Rihanna
5248,Flex (Ooh Ooh Ooh),Rich Homie Quan,2015,49,Flex (Ooh Ooh Ooh) | Rich Homie Quan
5249,Nasty Freestyle,T-Wayne,2015,50,Nasty Freestyle | T-Wayne
5250,Only,"Nicki Minaj feat. Drake, Lil Wayne and Chris B...",2015,51,"Only | Nicki Minaj feat. Drake, Lil Wayne and ..."
5251,Elastic Heart,Sia,2015,52,Elastic Heart | Sia


In [26]:
## Querying the Spotify API for the details of every song in our test_df
test_df[['Spotify Artist', 'Spotify Song', 'Spotify ID']] = test_df['Query'].map(get_song_details).apply(pd.Series)




/Users/jcacho/anaconda3/envs/DSI/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [27]:
## Check results
test_df

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID
5247,B**** Better Have My Money,Rihanna,2015,48,B**** Better Have My Money | Rihanna,Rihanna,Bitch Better Have My Money,0NTMtAO2BV4tnGvw9EgBVq
5248,Flex (Ooh Ooh Ooh),Rich Homie Quan,2015,49,Flex (Ooh Ooh Ooh) | Rich Homie Quan,Rich Homie Quan,"Flex (Ooh, Ooh, Ooh)",2LLFl56eIFGFlnrplUC26f
5249,Nasty Freestyle,T-Wayne,2015,50,Nasty Freestyle | T-Wayne,T-Wayne,Nasty Freestyle,0PT7nlpo11hYYyfnBgtilT
5250,Only,"Nicki Minaj feat. Drake, Lil Wayne and Chris B...",2015,51,"Only | Nicki Minaj feat. Drake, Lil Wayne and ...",Backing Force,Only (Originally Performed by Nicki Minaj feat...,0CSrlx5BND6JjYLhhESiHP
5251,Elastic Heart,Sia,2015,52,Elastic Heart | Sia,Sia,Elastic Heart,5S9lfZu1YF8xKkgSsqL2Bd
5252,Cool For The Summer,Demi Lovato,2015,53,Cool For The Summer | Demi Lovato,Demi Lovato,Cool for the Summer,2zFF6jG5hQArbzcXz3KUWk
5253,Renegades,X Ambassadors,2015,54,Renegades | X Ambassadors,X Ambassadors,Renegades,0fYVliAYKHuPmECRs1pbRf
5254,I Don't Mind,Usher feat. Juicy J,2015,55,I Don't Mind | Usher feat. Juicy J,Usher,I Don't Mind (feat. Juicy J),7aXuop4Qambx5Oi3ynsKQr
5255,Love Me Harder,Ariana Grande and The Weeknd,2015,56,Love Me Harder | Ariana Grande and The Weeknd,Sing2Piano,Love Me Harder (Lower Key) [Originally Perform...,418oDHYlbPlh5sY5HoBKbR
5256,Wildest Dreams,Taylor Swift,2015,57,Wildest Dreams | Taylor Swift,Taylor Swift,Wildest Dreams,59HjlYCeBsxdI0fcm3zglw


In [28]:
## Now lets try it for real on the top100 dataset
## This can take up to half an hour.
top100[['Spotify Artist', 'Spotify Song', 'Spotify ID']] = top100['Query'].map(get_song_details).apply(pd.Series)
backup_top100 = top100

In [29]:
# Check for null values (i.e. songs that were not found)
top100[top100['Spotify ID'].isnull()]

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID
491,Little Ole Man (Uptight-Everything's Alright),Bill Cosby,1967,92,Little Ole Man (Uptight-Everything's Alright) ...,NaN,NaN,NaN
574,Shoo-Bee-Doo-Be-Doo-Da-Day,Stevie Wonder,1968,75,Shoo-Bee-Doo-Be-Doo-Da-Day | Stevie Wonder,NaN,NaN,NaN
718,Thank You (Fallettin Me Be Mice Elf Again),Sly and The Family Stone,1970,19,Thank You (Fallettin Me Be Mice Elf Again) | S...,NaN,NaN,NaN
821,Uncle Albert-Admiral Halsey,Paul and Linda McCartney,1971,22,Uncle Albert-Admiral Halsey | Paul and Linda M...,NaN,NaN,NaN
938,Day Atter Day,Badfinger,1972,39,Day Atter Day | Badfinger,NaN,NaN,NaN
985,Don't Say You Don't Rememeer,Beverly Bremers,1972,86,Don't Say You Don't Rememeer | Beverly Bremers,NaN,NaN,NaN
997,Walking In The Rain With The One I Love,Love Unlimited,1972,98,Walking In The Rain With The One I Love | Love...,NaN,NaN,NaN
1265,One Man Woman-One Woman Man,Paul Anka and Odia Coates,1975,66,One Man Woman-One Woman Man | Paul Anka and Od...,NaN,NaN,NaN
1490,Whispering-Cherchez La Femme-C'est Si Bon,Dr. Buzzard's Original Savannah Band,1977,91,Whispering-Cherchez La Femme-C'est Si Bon | Dr...,NaN,NaN,NaN
1960,Owe Hundred Ways,Quincy Jones,1982,61,Owe Hundred Ways | Quincy Jones,NaN,NaN,NaN


In [30]:
## Keep track of songs that weren't found
missing_vals = top100[top100['Spotify ID'].isnull()]

In [31]:
missing_vals

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID
491,Little Ole Man (Uptight-Everything's Alright),Bill Cosby,1967,92,Little Ole Man (Uptight-Everything's Alright) ...,NaN,NaN,NaN
574,Shoo-Bee-Doo-Be-Doo-Da-Day,Stevie Wonder,1968,75,Shoo-Bee-Doo-Be-Doo-Da-Day | Stevie Wonder,NaN,NaN,NaN
718,Thank You (Fallettin Me Be Mice Elf Again),Sly and The Family Stone,1970,19,Thank You (Fallettin Me Be Mice Elf Again) | S...,NaN,NaN,NaN
821,Uncle Albert-Admiral Halsey,Paul and Linda McCartney,1971,22,Uncle Albert-Admiral Halsey | Paul and Linda M...,NaN,NaN,NaN
938,Day Atter Day,Badfinger,1972,39,Day Atter Day | Badfinger,NaN,NaN,NaN
985,Don't Say You Don't Rememeer,Beverly Bremers,1972,86,Don't Say You Don't Rememeer | Beverly Bremers,NaN,NaN,NaN
997,Walking In The Rain With The One I Love,Love Unlimited,1972,98,Walking In The Rain With The One I Love | Love...,NaN,NaN,NaN
1265,One Man Woman-One Woman Man,Paul Anka and Odia Coates,1975,66,One Man Woman-One Woman Man | Paul Anka and Od...,NaN,NaN,NaN
1490,Whispering-Cherchez La Femme-C'est Si Bon,Dr. Buzzard's Original Savannah Band,1977,91,Whispering-Cherchez La Femme-C'est Si Bon | Dr...,NaN,NaN,NaN
1960,Owe Hundred Ways,Quincy Jones,1982,61,Owe Hundred Ways | Quincy Jones,NaN,NaN,NaN


In [32]:
## Drop NAs and reset index
top100.dropna(inplace=True)
top100.reset_index(drop=True, inplace=True)
top100.shape

(5271, 8)

In [33]:
## Let's compare our original song details to the queried song details to make sure we're grabbing the right songs
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [34]:
similar('Jimmy Gilmer and The Fireballs', 'Jimmy Gilmer & The Fireballs')

0.9310344827586207

In [35]:
top100['Similarity Score'] = similar(top100['Artist'], top100['Spotify Artist'])

In [36]:
## Checking for mislabeling of observations
for row in range(0, len(top100)):
    try:
        top100.loc[row, 'Similarity Score'] = similar(top100.loc[row, 'Artist'], top100.loc[row,'Spotify Artist'])
    except:
        top100.loc[row, 'Similarity Score'] = np.nan

In [37]:
top100.head()

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID,Similarity Score
0,Sugar Shack,Jimmy Gilmer and The Fireballs,1963,1,Sugar Shack | Jimmy Gilmer and The Fireballs,Jimmy Gilmer & The Fireballs,Sugar Shack,2nLLenueHlqs60IcDn9lan,0.931034
1,Surfin' U.S.A.,Beach Boys,1963,2,Surfin' U.S.A. | Beach Boys,The Beach Boys,Surfin' U.S.A. - Remastered,0wz1LjDb9ZNEYwOmDJ3Q4b,0.833333
2,The End Of The World,Skeeter Davis,1963,3,The End Of The World | Skeeter Davis,Skeeter Davis,The End of the World,5DTOOkooKFUvWj1XQTFa09,1.000000
3,Rhythm Of The Rain,Cascades,1963,4,Rhythm Of The Rain | Cascades,The Cascades,Rhythm Of The Rain,7GHf8dQ2wA71wYOuRsQCZP,0.800000
4,He's So Fine,Chiffons,1963,5,He's So Fine | Chiffons,The Chiffons,He's So Fine - Remastered 2000,2Bs1Lw9LX8hBm70t1iGdlh,0.800000


In [38]:
top100.tail()

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID,Similarity Score
5266,El Perdon (Forgiveness),Nicky Jam and Enrique Iglesias,2015,96,El Perdon (Forgiveness) | Nicky Jam and Enriqu...,Nicky Jam,El Perdón - Forgiveness,7hOfyNuj2dLc0HAEYVDIYS,0.461538
5267,She Knows,Ne-Yo feat. Juicy J,2015,97,She Knows | Ne-Yo feat. Juicy J,Backing Force,She Knows (Originally Performed by Ne-yo feat....,6UkElkPH4OGwh43eOTusZf,0.062500
5268,Night Changes,One Direction,2015,98,Night Changes | One Direction,One Direction,Night Changes,5O2P9iiztwhomNh8xkR9lJ,1.000000
5269,Back To Back,Drake,2015,99,Back To Back | Drake,Drake,Back To Back,5lFDtgWsjRJu8fPOAyJIAK,1.000000
5270,How Deep Is Your Love,Calvin Harris and Disciples,2015,100,How Deep Is Your Love | Calvin Harris and Disc...,Calvin Harris,How Deep Is Your Love - Calvin Harris & R3hab ...,6rMwYjoFvxKiaEH4If8ZpZ,0.650000


In [39]:
##Drop all songs that are karaoke versions
karaoke_songs = top100[top100['Spotify Artist'].str.contains('Karaoke')].index
top100.drop(karaoke_songs, inplace=True)

In [40]:
## 0.3 similarity score seems to be a good cut-off for songs that should be discarded
diff_names = top100[(top100['Similarity Score']<0.3) | (top100['Similarity Score'].isna())]
top100.drop(diff_names.index, inplace=True)

In [41]:
top100.shape

(4745, 9)

In [42]:
top100.reset_index(drop=True, inplace=True)

In [43]:
## Now lets grab the acoustic features of each song in our top100 dataset
top100_id = top100['Spotify ID']
audio_features = pd.DataFrame()
for song_id in top100_id:
    features = pd.DataFrame(sp.audio_features(song_id))
    audio_features = audio_features.append(features, ignore_index=True)

retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs


/Users/jcacho/anaconda3/envs/DSI/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [44]:
# Check results...
audio_features.head()

,0,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,NaN,0.225,https://api.spotify.com/v1/audio-analysis/2nLL...,0.792,123693.0,0.804,2nLLenueHlqs60IcDn9lan,0.000005,7.0,0.065,-7.484,1.0,0.0659,132.852,4.0,https://api.spotify.com/v1/tracks/2nLLenueHlqs...,audio_features,spotify:track:2nLLenueHlqs60IcDn9lan,0.879
1,NaN,0.661,https://api.spotify.com/v1/audio-analysis/0wz1...,0.550,149373.0,0.854,0wz1LjDb9ZNEYwOmDJ3Q4b,0.000000,3.0,0.112,-5.968,1.0,0.0360,159.231,4.0,https://api.spotify.com/v1/tracks/0wz1LjDb9ZNE...,audio_features,spotify:track:0wz1LjDb9ZNEYwOmDJ3Q4b,0.965
2,NaN,0.936,https://api.spotify.com/v1/audio-analysis/5DTO...,0.232,157573.0,0.216,5DTOOkooKFUvWj1XQTFa09,0.000005,10.0,0.246,-10.674,1.0,0.0314,94.268,3.0,https://api.spotify.com/v1/tracks/5DTOOkooKFUv...,audio_features,spotify:track:5DTOOkooKFUvWj1XQTFa09,0.432
3,NaN,0.627,https://api.spotify.com/v1/audio-analysis/7GHf...,0.659,146360.0,0.660,7GHf8dQ2wA71wYOuRsQCZP,0.000217,0.0,0.136,-8.337,1.0,0.0441,116.634,4.0,https://api.spotify.com/v1/tracks/7GHf8dQ2wA71...,audio_features,spotify:track:7GHf8dQ2wA71wYOuRsQCZP,0.672
4,NaN,0.487,https://api.spotify.com/v1/audio-analysis/2Bs1...,0.558,112520.0,0.633,2Bs1Lw9LX8hBm70t1iGdlh,0.000000,7.0,0.129,-3.265,1.0,0.0329,72.494,4.0,https://api.spotify.com/v1/tracks/2Bs1Lw9LX8hB...,audio_features,spotify:track:2Bs1Lw9LX8hBm70t1iGdlh,0.871


In [45]:
## Merge the top 100 acoustic features with the original top100 dataset
top100_details = pd.merge(top100, audio_features, left_on="Spotify ID", right_on="id")
top100_details.head()

,Song Title,Artist,Year,Position,Query,Spotify Artist,Spotify Song,Spotify ID,Similarity Score,0,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,Sugar Shack,Jimmy Gilmer and The Fireballs,1963,1,Sugar Shack | Jimmy Gilmer and The Fireballs,Jimmy Gilmer & The Fireballs,Sugar Shack,2nLLenueHlqs60IcDn9lan,0.931034,NaN,...,0.065,-7.484,1.0,0.0659,132.852,4.0,https://api.spotify.com/v1/tracks/2nLLenueHlqs...,audio_features,spotify:track:2nLLenueHlqs60IcDn9lan,0.879
1,Surfin' U.S.A.,Beach Boys,1963,2,Surfin' U.S.A. | Beach Boys,The Beach Boys,Surfin' U.S.A. - Remastered,0wz1LjDb9ZNEYwOmDJ3Q4b,0.833333,NaN,...,0.112,-5.968,1.0,0.0360,159.231,4.0,https://api.spotify.com/v1/tracks/0wz1LjDb9ZNE...,audio_features,spotify:track:0wz1LjDb9ZNEYwOmDJ3Q4b,0.965
2,The End Of The World,Skeeter Davis,1963,3,The End Of The World | Skeeter Davis,Skeeter Davis,The End of the World,5DTOOkooKFUvWj1XQTFa09,1.000000,NaN,...,0.246,-10.674,1.0,0.0314,94.268,3.0,https://api.spotify.com/v1/tracks/5DTOOkooKFUv...,audio_features,spotify:track:5DTOOkooKFUvWj1XQTFa09,0.432
3,Rhythm Of The Rain,Cascades,1963,4,Rhythm Of The Rain | Cascades,The Cascades,Rhythm Of The Rain,7GHf8dQ2wA71wYOuRsQCZP,0.800000,NaN,...,0.136,-8.337,1.0,0.0441,116.634,4.0,https://api.spotify.com/v1/tracks/7GHf8dQ2wA71...,audio_features,spotify:track:7GHf8dQ2wA71wYOuRsQCZP,0.672
4,He's So Fine,Chiffons,1963,5,He's So Fine | Chiffons,The Chiffons,He's So Fine - Remastered 2000,2Bs1Lw9LX8hBm70t1iGdlh,0.800000,NaN,...,0.129,-3.265,1.0,0.0329,72.494,4.0,https://api.spotify.com/v1/tracks/2Bs1Lw9LX8hB...,audio_features,spotify:track:2Bs1Lw9LX8hBm70t1iGdlh,0.871


In [46]:
top100_details.columns

Index([      'Song Title',           'Artist',             'Year',
               'Position',            'Query',   'Spotify Artist',
           'Spotify Song',       'Spotify ID', 'Similarity Score',
                        0,     'acousticness',     'analysis_url',
           'danceability',      'duration_ms',           'energy',
                     'id', 'instrumentalness',              'key',
               'liveness',         'loudness',             'mode',
            'speechiness',            'tempo',   'time_signature',
             'track_href',             'type',              'uri',
                'valence'],
      dtype='object')

In [47]:
## Reordering columns to match the `songs` dataframe
top100_details = top100_details[['id', 'Song Title', 'Artist', 'acousticness', 'danceability',
                                 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
                                 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature',
                                 'valence', 'Year', 'Position']]

In [48]:
top100_details.columns

Index(['id', 'Song Title', 'Artist', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'Year', 'Position'],
      dtype='object')

In [49]:
top100_details.columns = ['id', 'song', 'artist', 'acousticness', 'danceability',
                 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
                  'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 
                  'valence', 'year', 'position']

In [50]:
songs.columns

Index(['id', 'song', 'artist', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'year'],
      dtype='object')

In [51]:
top100_details.columns

Index(['id', 'song', 'artist', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'year',
       'position'],
      dtype='object')

In [52]:
top100_details.head()

,id,song,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year,position
0,2nLLenueHlqs60IcDn9lan,Sugar Shack,Jimmy Gilmer and The Fireballs,0.225,0.792,123693.0,0.804,0.000005,7.0,0.065,-7.484,1.0,0.0659,132.852,4.0,0.879,1963,1
1,0wz1LjDb9ZNEYwOmDJ3Q4b,Surfin' U.S.A.,Beach Boys,0.661,0.550,149373.0,0.854,0.000000,3.0,0.112,-5.968,1.0,0.0360,159.231,4.0,0.965,1963,2
2,5DTOOkooKFUvWj1XQTFa09,The End Of The World,Skeeter Davis,0.936,0.232,157573.0,0.216,0.000005,10.0,0.246,-10.674,1.0,0.0314,94.268,3.0,0.432,1963,3
3,7GHf8dQ2wA71wYOuRsQCZP,Rhythm Of The Rain,Cascades,0.627,0.659,146360.0,0.660,0.000217,0.0,0.136,-8.337,1.0,0.0441,116.634,4.0,0.672,1963,4
4,2Bs1Lw9LX8hBm70t1iGdlh,He's So Fine,Chiffons,0.487,0.558,112520.0,0.633,0.000000,7.0,0.129,-3.265,1.0,0.0329,72.494,4.0,0.871,1963,5


In [53]:
## Fill in the position column in the `songs` dataset with NaNs
songs['position'] = np.nan

In [54]:
##Drop songs that exist in top100 from the `songs` dataset
duplicates = songs[songs['id'].isin(top100_details['id'])]
songs.drop(duplicates.index, inplace=True)

In [55]:
##Combine the original `songs` dataset with the `top100` dataset
tracks = pd.concat([songs, top100_details])
tracks.reset_index(drop=True, inplace=True)
tracks.head()

,id,song,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year,position
0,4TTV7EcfroSLWzXRY6gLv6,Alexander Hamilton,Original Broadway Cast Recording,0.524,0.609,236738.0,0.435,0.0,11.0,0.1180,-7.862,0.0,0.284,131.998,4.0,0.563,2015,NaN
1,6dr7ekfhlbquvsVY8D7gyk,"Aaron Burr, Sir",Original Broadway Cast Recording,0.271,0.646,156631.0,0.438,0.0,8.0,0.0945,-10.837,1.0,0.808,160.433,4.0,0.524,2015,NaN
2,4cxvludVmQxryrnx1m9FqL,My Shot,Original Broadway Cast Recording,0.150,0.829,333154.0,0.649,0.0,8.0,0.0904,-6.764,1.0,0.317,90.936,4.0,0.569,2015,NaN
3,0NJWhm3hUwIZSy5s0TGJ8q,The Story of Tonight,Original Broadway Cast Recording,0.835,0.407,91981.0,0.237,0.0,9.0,0.6480,-10.579,1.0,0.053,95.111,3.0,0.612,2015,NaN
4,71X7bPDljJHrmEGYCe7kQ8,The Schuyler Sisters,Original Broadway Cast Recording,0.186,0.740,186543.0,0.727,0.0,2.0,0.6270,-6.253,1.0,0.278,101.408,4.0,0.792,2015,NaN


In [56]:
tracks.shape

(305929, 18)

In [57]:
tracks['top100'] = tracks['position'].map(lambda x: 0 if pd.isnull(x) else 1)

In [58]:
## Total number of tracks in the top100
tracks['top100'].sum()

5201

In [59]:
##Removing songs that are duplicated because they charted the Top 100 for multiple years
duplicates = tracks[tracks.duplicated(subset='id')]
tracks.drop(duplicates.index, inplace=True)

In [60]:
## Total number of tracks in the top100
tracks['top100'].sum()

4517

In [61]:
##Duplicates defined by song name and artist name. Frame is ordered to keep the row with top100 indicator. 
duplicates = tracks[tracks.duplicated(subset=['song', 'artist'], keep=False)].sort_values(by=['song', 'top100'], ascending=False)
duplicates

,id,song,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year,position,top100
73796,1ieiOxk5Je0GJBIscCB4H8,レット・イット・ゴー~ありのままで~ - Japanese Version,Various Artists,0.572000,0.531,224480.0,0.5030,0.000000,5.0,0.1240,-6.811,0.0,0.0286,137.102,4.0,0.3160,2014,NaN,0
73823,3tiQdnBlEksucRa8i1lcNi,レット・イット・ゴー~ありのままで~ - Japanese Version,Various Artists,0.753000,0.582,224600.0,0.4530,0.000000,8.0,0.1250,-6.110,1.0,0.0290,136.960,4.0,0.3230,2014,NaN,0
59299,1bBT65uYdVFeEi2rk6rKJT,Überlin,R.E.M.,0.000956,0.599,254533.0,0.7560,0.003080,7.0,0.2600,-4.682,0.0,0.0281,133.993,4.0,0.2580,2011,NaN,0
66878,0vyQzGo4ZhrOvp0zMkpyLP,Überlin,R.E.M.,0.001030,0.594,254613.0,0.7470,0.002440,7.0,0.3540,-4.670,0.0,0.0273,133.986,4.0,0.2500,2011,NaN,0
71320,1VlS8vBLvIHxGg8WpBX20o,Él Nunca Te Olvida,Marco Antonio Solis,0.161000,0.614,231733.0,0.6740,0.000137,0.0,0.1910,-4.564,1.0,0.0232,99.184,4.0,0.4640,2010,NaN,0
136801,4dZfmVr4zqUYVB2GQnwAeY,Él Nunca Te Olvida,Marco Antonio Solis,0.127000,0.613,231733.0,0.6600,0.000230,0.0,0.1150,-4.797,1.0,0.0230,99.210,4.0,0.4360,2011,NaN,0
112304,3bcKL0nHWIxWhlGswyTH60,¿Quién Te Dijo Eso?,Luis Fonsi,0.109000,0.519,274840.0,0.5890,0.000000,5.0,0.1290,-5.511,1.0,0.0280,133.930,4.0,0.3510,2006,NaN,0
137053,159yUhKZg2ZAdt0HknTqDr,¿Quién Te Dijo Eso?,Luis Fonsi,0.115000,0.525,271600.0,0.5790,0.000000,5.0,0.2090,-6.118,1.0,0.0281,133.956,4.0,0.3440,2003,NaN,0
71315,1Agu4u8a8Qm28zq5JtwaVv,¿A Dónde Vamos A Parar?,Marco Antonio Solis,0.492000,0.413,228920.0,0.5110,0.003010,0.0,0.1240,-4.958,1.0,0.0313,72.023,4.0,0.2020,2010,NaN,0
136786,4fDMVI79jXH7S26uawPeXL,¿A Dónde Vamos A Parar?,Marco Antonio Solis,0.446000,0.386,229253.0,0.4940,0.005890,0.0,0.1200,-4.716,1.0,0.0286,144.130,4.0,0.1910,2011,NaN,0


In [62]:
##The records to be dropped
duplicates = duplicates[duplicates.duplicated(subset=['song', 'artist'])]
tracks.drop(duplicates.index, inplace=True)

In [63]:
##Making sure the number of hit songs hasn't changed
tracks['top100'].sum()

4517

In [64]:
tracks.shape

(291631, 19)

In [65]:
## Write the final dataset to CSV
tracks.to_csv('./data/tracks.csv')